In [2]:
import pandas as pd
import numpy as np
import pymssql
import pyodbc

In [3]:
mssql_conn = pymssql.connect(server='epdm', user='sa', password='Pdmepdme1!',database='ePDM_Elcam')
odbc_conn = pyodbc.connect('Driver={IBM i Access ODBC Driver}; '
       'SYSTEM = 192.168.0.161;'
        'Hostname=M3; '
        'Port=21; '
        'Protocol=TCPIP; '
        'Database=M3; '
        'UID=pud; '
        'PWD = Pud142;'
        ,autocommit=True)

In [30]:
#test connection
query='SELECT Filename, LockPath, LockDomain, LockDate, Username,FullName, Email from [ePDM_Elcam].[dbo].[Documents] INNER JOIN [ePDM_Elcam].[dbo].[Users] on [ePDM_Elcam].[dbo].[Documents].UserID = [ePDM_Elcam].[dbo].[Users].UserID WHERE LockDate IS NOT NULL AND CurrentStatusid IS NOT NULL AND Deleted=0 ORDER BY Username ASC'
query2='SELECT distinct(MMITNO) FROM M3FDBPRD.MITMAS m '
all_items=pd.read_sql(query2,odbc_conn)
files = pd.read_sql(query, mssql_conn)
files.sort_values(by='LockDate', ascending=False, inplace=True)
#find if item number '015495DOL      ' is in the list of all items





c:\Users\matan.s\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\matan.s\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [33]:
def find_file(item):
    item=item.strip()
    query =f"select Filename from Documents d where Filename like '%{item}%'"
    result = pd.read_sql(query, mssql_conn)
    print(result)
find_file('7800B')

c:\Users\matan.s\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


                                             Filename
0            2201307801- 7800b-בדיקות תחילת ייצור.pdf
1       2201319001 בדיקות תחילת ייצור -7800B- לוט.pdf
2                   580302 560302+7800b+530002.sldasm
3                   580304 560307+7800b+530002.sldasm
4                   580306 560306+7800b+530002.sldasm
5                   580308 560301+7800b+530002.SLDPRT
6                   580402 560402+7800b+530002.sldasm
7                   580406 560406+7800b+530002.sldasm
8                    581408 561442+7800b+761eb.sldasm
9                    581431 561480+7800b+761eb.sldasm
10                   581432 561481+7800b+761eb.sldasm
11                  581456 521418+511000+7800b.sldasm
12                   582122 562132+761en+7800b.sldasm
13                 582482  562433+7800b+530002.sldasm
14                                     7800B x.sldprt
15                              7800B(for FAT).sldprt
16                                         7800B..PDF
17                          

In [31]:

res ={}
for i in range(len(files)):
    row=files.iloc[i]
    if row['Username'] not in res:
        res[row['Username']] ={'email':row['Email'],'fullname':row['FullName'],'count':0}
    res[row['Username']]['count']+=1
#sort by count
res = sorted(res.items(), key=lambda x: x[1]['count'], reverse=True)
username=[]
fullname=[]
email=[]
count_files=[]
for k in res:
    # print(k)
    username.append(k[0])
    fullname.append(k[1]['fullname'])
    email.append(k[1]['email'])
    count_files.append(k[1]['count'])
df = pd.DataFrame({'username':username,'fullname':fullname,'email':email,'count_files':count_files})
df



,username,fullname,email,count_files
0,hen.b,Hen Balik,hen.b@elcam.co.il,71
1,ayelet.ts,Ayelet Tal Shahar,ayelet.ts@elcam.co.il,54
2,sergei.h,Sergei Holodovsky,sergei.h@elcam.co.il,49
3,dima.a,Dima Ali,dima.a@elcam.co.il,27
4,yair.e,Yair Elbaz,yair.e@elcam.co.il,23
...,...,...,...,...
62,hany.h,Hany Haj,hany.h@elcam.co.il,1
63,Liana.b,Liana Bruder,Liana.b@elcam.co.il,1
64,lev.g,Lev Gens,lev.g@elcam.co.il,1
65,sharbel.a,Sharbel Aboud,sharbel.a@elcam.co.il,1


In [9]:
df2=pd.read_csv("kimaat.moshlam.csv")
#remove all the rows that have no email address
df2=df2[df2['EmailAddress'].notnull()]
#remove all the rows that the email address is not end with @elcam.co.il
df2=df2[df2['EmailAddress'].str.endswith('@elcam.co.il')]
#convert all the email address to lower case
df2['EmailAddress']=df2['EmailAddress'].str.lower()
#find the rows that the display name is Gal Einhorn
df2[df2['ManagerEmail']=='amotz@elcam.co.il']


,DisplayName,EmailAddress,ManagerEmail
707,Vanya Dobrovolsky,vanya.d@elcam.co.il,amotz@elcam.co.il
755,Idan Naaman,idan.n@elcam.co.il,amotz@elcam.co.il
922,Ilan Kachler,ilan.k@elcam.co.il,amotz@elcam.co.il
938,Oded Benedik,oded.b@elcam.co.il,amotz@elcam.co.il
1116,Ziv Cohen,ziv.c@elcam.co.il,amotz@elcam.co.il
1207,Ohad Gottesman,ohad.g@elcam.co.il,amotz@elcam.co.il


In [1]:


message = Message(From="me@example.com",
                  To="you@example.com",
                  charset="utf-8")
message.Subject = "An HTML Email"
message.Html = """This email uses <strong>HTML</strong>!"""
message.Body = """This is alternate text."""

sender = Mailer('smtp.example.com')
sender.send(message)



ModuleNotFoundError: No module named 'mailer'